In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipe=Pipeline([
           ('scaler',StandardScaler()),
           ('imp',SimpleImputer(strategy='mean',missing_values=np.nan)),
])

In [ ]:
train_set = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test_set = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
values=test_set['id']
test_set=test_set.drop(['id'],axis=1).copy()

In [ ]:
X_train=train_set.drop(['id','claim'],axis=1).copy()
Y_train=train_set['claim'].copy()
Y_train

In [ ]:
X_train= pd.DataFrame(columns=X_train.columns, data=num_pipe.fit_transform(X_train))
test_set = pd.DataFrame(columns=test_set.columns, data=num_pipe.transform(test_set))

In [ ]:
import catboost
from catboost import CatBoostClassifier

In [ ]:
# best_params={'iterations': 15585, 
#     'objective': 'CrossEntropy', 
#     'bootstrap_type': 'Bernoulli', 
#     'od_wait': 1144, 
#     'learning_rate': 0.023575206684596582, 
#     'reg_lambda': 36.30433203563295, 
#     'random_strength': 43.75597655616195, 
#     'depth': 7, 
#     'min_data_in_leaf': 11, 
#     'leaf_estimation_iterations': 1, 
#     'subsample': 0.8227911142845009,
#     'task_type' : 'GPU',
#     'devices' : '0',
#     'verbose' : 0}

In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import roc_curve,auc
# from sklearn.model_selection import StratifiedKFold

# kf=StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
# pred=[]
# for folds,(idx_train,idx_valid) in enumerate(kf.split(X_train,Y_train)):

#   x_train, y_train = X_train.iloc[idx_train], Y_train.iloc[idx_train]
#   x_valid, y_valid = X_train.iloc[idx_valid], Y_train.iloc[idx_valid]


#   model=CatBoostClassifier(**best_params)
#   model.fit(x_train,y_train)

#   prob=model.predict_proba(x_valid)[:,1]
#   fpr, tpr, _ = roc_curve(y_valid, prob)
#   score = auc(fpr, tpr)

#   print(f'Fold:{folds+1} Score:{score}')
#   preds=model.predict_proba(test_set)[:,1]
#   pred.append(preds)

In [ ]:
from sklearn.model_selection import cross_val_score,GridSearchCV
params={'iterations':100,'learning_rate':0.5,'objective':'Logloss'}
model=CatBoostClassifier(**params)
model.fit(X_train,Y_train)

In [ ]:
pred=model.predict_proba(test_set)

In [ ]:
preds=pred[:,1]

In [ ]:
sub   = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
features = [col for col in train_set.columns if col not in ['claim', 'id']]

In [ ]:
output2=pd.DataFrame({'id':values,'claim':preds})
output2

In [ ]:
output2.to_csv('submission.csv', index=False)